In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import CsemigroupsCpp
from CsemigroupsCpp import *
import PyNormaliz
from PyNormaliz import *
import itertools
from scipy.spatial import ConvexHull

In [ ]:
# Calculo el menor cuboide que contenga al diamante.
# INPUT:
#   - d: Vértices del diamante.
# OUTPUT:
#   - Máximos en cada coordenada del cubo.
def Cube(d):
    dim = len(d[0])
    aux = []
    for i in range(dim):
        aux.append(sorted(d, key = lambda x: x[i])[-1][i])
    return(aux)

In [ ]:
class Csemigroup:
    def __init__(self,generators):
        self.gaps = None
        self.pf = None
        aux = self.__ComputeMinimalGenerators(generators)
        self.generators = list([list(x) for x in aux])
        self.generators.sort(key=lambda row: row[1:])
        coneSg = Cone(cone=self.generators)
        self.rays = coneSg.ExtremeRays()
        self.hyperplanes = coneSg.SupportHyperplanes()
        print(self.__IsCSemigroup())
        
    def __IsCSemigroup(self):
        if not axisAreSemigroup(self.generators,self.rays):
            raise Exception("This set does not generate a numerical semigroup.")
        diamondA = diamond(self.rays)
        hull = ConvexHull(diamondA)
        eqDiamond = [list(x) for x in hull.equations]
        boundDiamond = Cube(diamondA)
        it = itertools.product(*[range(i+1) for i in boundDiamond])
        return "raysSg"
        
    def __ComputeMinimalGenerators(self,generators):
        return computeMSG(generators)
        
    def GetGenerators(self):
        return self.generators
    def GetRays(self):
        return self.rays
    def GetHyperplanes(self):
        return self.hyperplanes
    def GetGaps(self):
        if self.gaps == None:
            return self.gaps
        else:
            pass
            self.gaps = computeGaps(self.generators,self.rays,self.hyperplanes)

In [ ]:
generators = [[3, 1], [7, 1], [8, 2], [9, 1], [9, 2], [10, 1], [10, 3], [10, 4], [11, 2], [13, 1], [14, 5], [15, 6], [17, 7], [19, 8], [21, 2], [24, 2], [25, 2], [26, 11], [27, 2], [28, 2], [29, 2], [44, 3], [45, 3], [61, 4], [62, 4], [76, 5], [77, 5], [93, 6], [108, 7], [43, 3], [59, 4], [60, 4], [92, 6], [42, 18], [56, 24], [63, 27], [70, 30]]
cs = Csemigroup(generators)

In [ ]:
cs.GetRays()

In [ ]:
cs.GetHyperplanes()

In [ ]:
-2*3+31*1

In [ ]:
type([[-0.39391929857916763, 0.9191450300180578, -0.0], [-0.06438227799796505, 0.9979253089684582, -2.543099980919619], [0.06438227799796505, -0.9979253089684582, -0.0], [0.39391929857916763, -0.9191450300180578, -10.373208195918082]][0][0])

In [2]:
pointBelongsDiamond([0,0],[[-0.39391929857916763, 0.9191450300180578, -0.0], [-0.06438227799796505, 0.9979253089684582, -2.543099980919619], [0.06438227799796505, -0.9979253089684582, -0.0], [0.39391929857916763, -0.9191450300180578, -10.373208195918082]])

False

In [ ]:
axisAreSemigroup(cs.GetGenerators(),cs.rays)